In [ ]:
!pip install matplotlib
!pip install scikit-learn
!pip install pyod

In [ ]:
import matplotlib.pyplot as plt
import sklearn.metrics
from pyod.utils.data import generate_data
from pyod.models.knn import KNN
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import numpy as np
from sklearn.metrics import balanced_accuracy_score

In [ ]:
def ex1(contamination=0.1):
    data = generate_data(n_train=400, n_test=100, n_features=2, contamination=contamination, random_state=23)
    train_data, test_data, train_labels, test_labels = data

    train_inliers = train_data[train_labels == 0]
    train_outliers = train_data[train_labels == 1]

    plt.scatter(train_inliers[:, 0], train_inliers[:, 1], c='blue', label='Train Inliers')
    plt.scatter(train_outliers[:, 0], train_outliers[:, 1], c='red', label='Train Outliers')

    plt.show()

    return train_data, test_data, train_labels, test_labels

ex1()